In [4]:
from datascience import *
import numpy as np
import matplotlib

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter("ignore")

## Review

In [5]:
ckd = Table.read_table('ckd.csv').relabeled('Blood Glucose Random', 'Glucose')
ckd.sample(3).show(3)

Age,Blood Pressure,Specific Gravity,Albumin,Sugar,Red Blood Cells,Pus Cell,Pus Cell clumps,Bacteria,Glucose,Blood Urea,Serum Creatinine,Sodium,Potassium,Hemoglobin,Packed Cell Volume,White Blood Cell Count,Red Blood Cell Count,Hypertension,Diabetes Mellitus,Coronary Artery Disease,Appetite,Pedal Edema,Anemia,Class
59,100,1.015,4,2,normal,normal,notpresent,notpresent,255,132,12.8,135,5.7,7.3,20,9800,3.9,yes,yes,yes,good,no,yes,1
40,80,1.025,0,0,normal,normal,notpresent,notpresent,140,10,1.2,135,5,15,48,10400,4.5,no,no,no,good,no,no,0
57,60,1.02,0,0,normal,normal,notpresent,notpresent,132,18,1.1,150,4.7,15.4,42,11000,4.5,no,no,no,good,no,no,0


In [6]:
ckd = ckd.select('Hemoglobin', 'Glucose', 'Class')
ckd.show(3)

Hemoglobin,Glucose,Class
11.2,117,1
9.5,70,1
10.8,380,1


In [7]:
ckd.sample(3).show(3)

Hemoglobin,Glucose,Class
15,109,0
14.3,100,0
16.9,100,0


### How do we evaluate our prediction machine?

#### Training and testing sets

- The training set and testing set need to be determined at random.
- In the following code, we are *permuting* the rows randomly and then extracting a test subset and training subset using `np.take()`. 

**Task**: Split the data in the `ckd` table into training and testing sets of (roughly) equal size.

In [8]:
halfway = round(ckd.num_rows/2)
halfway

79

In [9]:
shuffled_patients = ckd.sample(with_replacement=False)

In [10]:
training = shuffled_patients.take(np.arange(halfway))
testing = shuffled_patients.take(np.arange(halfway, ckd.num_rows))

In [11]:
training.num_rows

79

In [12]:
testing.num_rows

79

## New material

### How can we find the *k*-nearest neighbors?

- We will work with the `ckd` dataset
- Goal: predict whether an out-of-sample patient has the disease or not.
   - using the entire `ckd` dataset as training.

Here's our out-of-sample point ("Alice") for today.

In [13]:
alice = make_array(0,1.1)
alice

array([ 0. ,  1.1])

#### Detour: Rows and row objects

In [14]:
ckd.row(0)

Row(Hemoglobin=11.199999999999999, Glucose=117, Class=1)

In [15]:
type(ckd.row(0))

datascience.tables.Row

In [16]:
ckd.row(0).item(1)

117

In [17]:
make_array(ckd.row(0))

array([[11.199999999999999, 117, 1]], dtype=object)

Let's save the first training point (as a row object) into the name `patient`.

In [18]:
patient = ckd.drop('Class').row(0)
patient

Row(Hemoglobin=11.199999999999999, Glucose=117)

In [27]:
make_array(patient)

array([[11.199999999999999, 117]], dtype=object)

#### **Task**: Let's find a distance between `alice` and the first training point.

$$ \text{D} = \sqrt{(x_0-x_1)^2 + (y_0-y_1)^2} $$

In [24]:
(alice.item(0) - patient.item(0)) ** 2 + (alice.item(1) - patient.item(1)) ** 2 

13558.250000000002

In [25]:
#this is an array 
alice

array([ 0. ,  1.1])

In [26]:
#This is an array
make_array(patient)

array([[11.199999999999999, 117]], dtype=object)

**Discussion Question** [1 min] Is there a quicker way to do the part $(x_0-x_1)^2 + (y_0-y_1)^2$?

- *Hint*: do we have to access the individual elements using the `.item` method, or can we do something simpler?

In [28]:
alice-make_array(patient) 

array([[-11.199999999999999, -115.90000000000001]], dtype=object)

In [29]:
(alice-make_array(patient))**2

array([[125.43999999999998, 13432.810000000001]], dtype=object)

In [30]:
np.sum((alice-make_array(patient))**2)

13558.250000000002

In [31]:
np.sum((alice-make_array(patient))**2) ** 0.5

116.43989866021012

In [36]:
def distance(point1, point2):
    """Returns the distance between point1 and point2
    where each argument is an array 
    consisting of the coordinates of the point"""
    return np.sum((point1-point2)**2) ** 0.5

In [ ]:
distance(alice, make_array(patient))

In [37]:
def distance_from_alice(row):
    return distance(alice, make_array(row))

In [ ]:
distance_from_alice(ckd.drop('Class').row(0))                     

_______

#### **Task**: Calculate the Euclidean distance between `alice` and every point in the training set.

In [34]:
ckd.drop('Class').take(np.arange(5))

Hemoglobin,Glucose
11.2,117
9.5,70
10.8,380
5.6,157
7.7,173


**Discussion Question** [1 min]: What's a method we can use to help us?


In [38]:
distances = ckd.drop('Class').apply(distance_from_alice)

In [39]:
def all_distances(training, new_row):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.drop('Class')
    def distance_from_new_row(row):
        return distance(make_array(new_row), make_array(row))
    return attributes.apply(distance_from_new_row)

In [40]:
distances = all_distances(ckd.drop('Class'), alice)

In [41]:
distances

array([ 116.43989866,   69.55185116,  379.05388799,  156.00054487,
        172.0723685 ,   94.41001006,  263.19699846,   69.62190747,
        252.11874187,  162.19633165,  128.22332081,  132.3015495 ,
         75.23576277,  279.20281159,  209.5194979 ,  218.14804606,
        294.0439593 ,  117.4545444 ,  223.04712507,  127.16465704,
        117.514297  ,  104.49124365,  287.00874551,  272.02665311,
        121.55480246,  302.07907905,  101.27437978,  106.22476171,
        116.33060646,  238.08960498,   93.42601351,  128.15623278,
        251.14985566,  254.00492121,  252.13571742,  213.1788451 ,
        489.0352339 ,  162.09262784,  240.0920032 ,  213.10741423,
        105.25193585,  423.08766231,  174.92747068,  139.70758748,
         70.96625959,   82.44768038,  118.90151387,   98.75935399,
        120.66884436,  130.70677106,   90.98356995,   97.89187913,
        104.15190829,  130.66300165,  122.08935253,  117.92311054,
        117.02670635,  131.6896731 ,   96.9456033 ,  132.55885

______

#### **Task:** Augment the training data table with a column containing all the distances.

In [50]:
ckd.with_column('Distance from Alice', distances)

Hemoglobin,Glucose,Class,Distance from Alice
11.2,117,1,116.44
9.5,70,1,69.5519
10.8,380,1,379.054
5.6,157,1,156.001
7.7,173,1,172.072
9.8,95,1,94.41
12.5,264,1,263.197
10,70,1,69.6219
10.5,253,1,252.119
9.8,163,1,162.196


In [52]:
def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

In [53]:
ckd_with_distances = table_with_distances(ckd, alice)

_______

#### **Task**: Find the 5 nearest neighbors.

**Discussion Question** [1 min]: Think of two table methods that will help us return a *table* with the five nearest neighbors. 

In [54]:
ckd_with_distances.sort('Distance')

Hemoglobin,Glucose,Class,Distance
9.5,70,1,69.5519
10,70,1,69.6219
13.7,70,0,70.2488
17,70,0,70.9663
15.1,74,0,74.4474
7.1,76,1,75.2358
16.5,75,0,75.7196
16.8,75,0,75.7856
16.1,78,0,78.5673
17.6,79,0,79.8634


In [56]:
ckd_with_distances.sort('Distance').take(np.arange(5))

Hemoglobin,Glucose,Class,Distance
9.5,70,1,69.5519
10,70,1,69.6219
13.7,70,0,70.2488
17,70,0,70.9663
15.1,74,0,74.4474


In [57]:
def nearest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    nearest_neighbors_table = sorted_by_distance.take(np.arange(k))
    return nearest_neighbors_table

In [58]:
five_nearest_neighbors = nearest(ckd, alice, 5)
five_nearest_neighbors

Hemoglobin,Glucose,Class,Distance
9.5,70,1,69.5519
10,70,1,69.6219
13.7,70,0,70.2488
17,70,0,70.9663
15.1,74,0,74.4474


### How do we classify the point?

#### **Task**: Find the majority class (programatically).

##### Method 1

In [59]:
ones = five_nearest_neighbors.where('Class', are.equal_to(1)).num_rows

In [60]:
zeros = five_nearest_neighbors.where('Class', are.equal_to(0)).num_rows

In [61]:
if ones > zeros:
    print(1)
else:
    print(0)

0


In [62]:
def majority(nearest_neighbors_table):
    """1 if the majority of the "Class" column is 1s, and 0 otherwise."""
    ones = nearest_neighbors_table.where('Class', are.equal_to(1)).num_rows
    zeros = nearest_neighbors_table.where('Class', are.equal_to(0)).num_rows
    if ones > zeros:
        return 1
    else:
        return 0

In [63]:
majority(five_nearest_neighbors)

0

##### Method 2

**Discussion Question:** [2 min]

You are given the `five_nearest_neighbors` table. Write out on a piece of paper (or type on your notes app) a line(s) of code that will find the majority class and return it as a string/number, *regardless* of whether the classes are called 0 or 1, or whatever their names are!

In [64]:
five_nearest_neighbors.group('Class')

Class,count
0,3
1,2


In [65]:
five_nearest_neighbors.group('Class').sort('count', descending=True)

Class,count
0,3
1,2


In [66]:
five_nearest_neighbors.group('Class').sort('count', descending=True).column('Class')

array([0, 1])

In [67]:
five_nearest_neighbors.group('Class').sort('count', 
                                           descending=True).column('Class').item(0)

0

In [68]:
def majority(nearest_neighbors_table, class_name):
    return nearest_neighbors_table.group(class_name).sort('count', 
                                                         descending=True).column(class_name).item(0)

In [69]:
majority(five_nearest_neighbors, 'Class')

0

#### **Task:** Write a function to complete the entire algorithm!

In [70]:
five_nearest_neighbors = nearest(ckd, alice, 5)
majority(five_nearest_neighbors, 'Class')

0

In [71]:
def knn(training, class_name, new_point, k):
    nearest_neighbors_table = nearest(training, new_point, k)
    return majority(nearest_neighbors_table, class_name)

In [72]:
knn(ckd, 'Class', alice, 5)

0

#### Here are all the functions, in order, that made up this algorithm.

In [ ]:
def distance(point1, point2):
    """Returns the distance between point1 and point2
    where each argument is an array 
    consisting of the coordinates of the point"""
    return np.sum((point1-point2)**2) ** 0.5

def all_distances(training, new_row):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.drop('Class')
    def distance_from_new(row):
        return distance(make_array(new_row), make_array(row))
    return attributes.apply(distance_from_new)


def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))


def nearest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    nearest_neighbors_table = sorted_by_distance.take(np.arange(k))
    return nearest_neighbors_table


def majority(nearest_neighbors_table, class_name):
    return nearest_neighbors_table.group(class_name).sort('count', 
                                                         descending=True).column(class_name).item(0)


def knn(training, class_name, new_point, k):
    nearest_neighbors_table = nearest(training, new_point, k)
    return majority(nearest_neighbors_table, class_name)

____

**Challenge Task**  [1 min] : Perform $k$-NN using the entire `banknotes` dataset as training! Make up a test point like we did with `alice` for the `ckd` dataset. What will the Euclidean distance calculation look like on paper if you use all of the variables (except `"Class"`) as predictors?

In [74]:
banknotes = Table.read_table('banknote.csv')
banknotes.sample(6)

WaveletVar,WaveletSkew,WaveletCurt,Entropy,Class
0.29961,7.1328,-0.31475,-1.1828,0
-1.5851,-2.1562,1.7082,0.9017,1
1.9157,6.0816,0.23705,-2.0116,0
4.4682,2.2907,0.95766,0.83058,0
2.2634,-4.4862,3.6558,-0.61251,0
2.7206,9.0821,-3.3111,-0.96811,0
